In [1]:
# Archit Kulkarni, Adhithya Narayanan, Vibhu Ambil, Tyler Youngberg
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/best-attempt/forecast.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv


In [2]:
'''
Title: M5 Data Preprocessing
Author: Quinn Wang
Date: March 2020
Availability: https://www.kaggle.com/qcw171717/naive-baseline/
'''

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tqdm import tqdm
df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
price_df = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")
cal_df = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1]))
price_df["id"] = price_df["item_id"] + "_" + price_df["store_id"] + "_validation"
for i in range(1886, 1914):
    df["F_" + str(i)] = 0

'''end Quinn Wang'''

'end Quinn Wang'

In [3]:
import lightgbm as lgb

Data Preprocessing Steps:

1. Convert categorical observations to int format
2. Shorten training data to last 28 days before predictions (days 1858-1886)
3. Create validation data from days 1887-1913

In [4]:
temp = np.zeros((30490,), dtype=int)
k = 0
for i in df["state_id"]:
    if i == "CA":
        temp[k] = 0
    if i == "TX":
        temp[k] = 1
    if i == "WI":
        temp[k] = 2
    k+=1
df["state_id"] = pd.DataFrame(temp)

In [5]:
temp = np.zeros((30490,), dtype=int)
k = 0
for i in df["store_id"]:
    if i == "CA_1":
        temp[k] = 0
    elif i == "CA_2":
        temp[k] = 1
    elif i == "CA_3":
        temp[k] = 2
    elif i == "CA_4":
        temp[k] = 3
    elif i == "TX_1":
        temp[k] = 4
    elif i == "TX_2":
        temp[k] = 5
    elif i == "TX_3":
        temp[k] = 6
    elif i == "WI_1":
        temp[k] = 7
    elif i == "WI_2":
        temp[k] = 8
    elif i == "WI_3":
        temp[k] = 9
    k+=1
df["store_id"] = pd.DataFrame(temp)

In [6]:
temp = np.zeros((30490,), dtype=int)
k = 0
for i in df["cat_id"]:
    if i == "HOBBIES":
        temp[k] = 0
    if i == "FOODS":
        temp[k] = 1
    if i == "HOUSEHOLD":
        temp[k] = 2
    k+=1
df["cat_id"] = pd.DataFrame(temp) 

In [7]:
temp = np.zeros((30490,), dtype=int)
k = 0
for i in df["dept_id"]:
    if i == "HOBBIES_1":
        temp[k] = 0
    elif i == "HOBBIES_2":
        temp[k] = 1
    elif i == "FOODS_1":
        temp[k] = 2
    elif i == "FOODS_2":
        temp[k] = 3
    elif i == "FOODS_3":
        temp[k] = 4
    elif i == "HOUSEHOLD_1":
        temp[k] = 5
    elif i == "HOUSEHOLD_2":
        temp[k] = 6
    k+=1
df["dept_id"] = pd.DataFrame(temp)

In [8]:
validation = pd.DataFrame()
train = df
for x in range(1886, 1914):
    s = "d_" + str(x)
    f = "F_" + str(x)
    train = train.drop(columns = [s, f])
    validation[s] = df[s]

In [9]:
validation

,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,1,0,0,0,0,0,1,0,4,2,...,1,3,0,1,1,1,3,0,1,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,3,1,2,1,3,1,...,1,0,5,4,1,0,1,3,7,2
4,1,0,4,4,0,1,4,0,1,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,1,1,2,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,1,0,0,0,0,1,1,2,1,4,...,2,1,0,2,0,1,0,0,1,0
30488,3,0,0,0,0,0,1,1,2,1,...,0,0,1,0,0,1,0,3,1,3


In [10]:
train = train.drop(columns=["item_id", "id"])

In [11]:
ids = pd.DataFrame()
ids["dept_id"] = df["dept_id"]
ids["cat_id"] = df["cat_id"]
ids["store_id"] = df["store_id"]
ids["state_id"] = df["state_id"]

In [12]:
ids

,dept_id,cat_id,store_id,state_id
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
30485,4,1,9,2
30486,4,1,9,2
30487,4,1,9,2
30488,4,1,9,2


Now, we will be creating a few models to see which one works the best. We will have 5 time periods, the minimum being the last 28 days, and the maximum being the last 800 days.

In [13]:
train1 = pd.concat([ids, pd.DataFrame(train.values[:, -28:],columns=train.columns[-28:].tolist())], axis=1) # Last 28 days
train2 = pd.concat([ids, pd.DataFrame(train.values[:, -50:],columns=train.columns[-50:].tolist())], axis=1) # Last 50 days
train3 = pd.concat([ids, pd.DataFrame(train.values[:, -200:],columns=train.columns[-200:].tolist())], axis=1) # last 200 days
train4 = pd.concat([ids, pd.DataFrame(train.values[:, -400:],columns=train.columns[-400:].tolist())], axis=1) # last 400 days
train5 = pd.concat([ids, pd.DataFrame(train.values[:, -800:],columns=train.columns[-800:].tolist())], axis=1) # last 800 days
t = [train1, train2, train3, train4, train5]

In [14]:
# merge ids dataframe with sales data
pd.concat([ids, train1], axis=1)

,dept_id,cat_id,store_id,state_id,dept_id,cat_id,store_id,state_id,d_1858,d_1859,...,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885
0,0,0,0,0,0,0,0,0,0,2,...,3,1,3,1,2,2,0,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,4,2,1,4,1,3,5,0,6,6
4,0,0,0,0,0,0,0,0,1,2,...,3,2,2,2,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,4,1,9,2,4,1,9,2,0,0,...,0,0,0,0,0,0,0,0,0,0
30486,4,1,9,2,4,1,9,2,0,0,...,0,0,0,0,0,0,0,0,0,0
30487,4,1,9,2,4,1,9,2,1,2,...,2,0,0,2,0,0,0,2,0,1
30488,4,1,9,2,4,1,9,2,2,2,...,1,1,1,1,0,2,1,1,2,4


In [15]:
from sklearn.metrics import mean_squared_error as mse
import math
def learner(t_data, v_data, params):
    temp = t_data.copy() # so that t_data is not changed
    for x in range(1886, 1914):
        d = pd.DataFrame()
        
        # setup for training
        t = temp.values
        v = v_data.values[:, x - 1886]
        lgbm_data = lgb.Dataset(t, label=v, 
                                feature_name=temp.columns.tolist(), 
                                categorical_feature=['dept_id', 'cat_id', 'store_id', 'state_id'])

        # train data
        gbt = lgb.train(params, lgbm_data) 

        s = "d_" + str(x) # string name for day in format d_xxxx
        x = pd.DataFrame(gbt.predict(t))

        # add new data to training for next iteration
        temp[s] = x
        
    forecast = pd.DataFrame(df["id"])
    for i in range(1886, 1914):
        d = "d_" + str(i)
        f = "F" + str(i-1885)
        forecast[f] = temp[d]
    print("RMSE = " + str(math.sqrt(mse(v_data.values, forecast.drop(columns=["id"]).values))))
    f2 = forecast.copy()
    f2["id"] = f2["id"].apply(lambda x : x.replace('validation', 'evaluation'))
    forecast = forecast.append(f2).reset_index(drop=True)
    return forecast

In [16]:
import warnings
warnings.simplefilter("ignore")

In [17]:
params = {
   'task': 'train',
   'boosting_type': 'gbdt',
   'objective': 'regression',
   'metric': 'rmse',
   'learning_rate': .1,
   'num_iterations': 75
}
for k in range(0, 5):
    f = learner(t[k], validation, params)
    name = "gbt-reg-75_" + str(k+1) + ".csv"
    f.to_csv(name, index=False)

RMSE = 1.5016115672411605
RMSE = 1.4554905765745727
RMSE = 1.321637005017294
RMSE = 1.2829561364089397
RMSE = 1.2530613467046605
